In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import re
import glob
import json
from pprint import pprint
from collections import ChainMap

import jq
import requests
from pyspark.sql import SparkSession, functions as F
from IPython.display import HTML

spark = SparkSession.builder.appName('test').getOrCreate()
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [3]:
SURVEY_ID = 284955270  # Test Survey

## Survey Overview
This section assumes you have a SurveyMonkey Access Token (`ACCESS_TOKEN` in `.env`)

To run notebook on Test Survey skip this section.

In [4]:
import os
import dotenv

dotenv.load_dotenv()
TOKEN = os.getenv("ACCESS_TOKEN")

headers = {
    'Authorization': f'Bearer {TOKEN}',
    'Content-Type': 'application/json'
}
response = requests.get(
    'https://api.surveymonkey.com/v3/surveys',
    headers=headers)
response.raise_for_status()

surveys = (
    jq
    .compile('.data | map({name: .id, value: .title}) | from_entries')
    .input(response.json())
    .all()
)
surveys = dict(ChainMap(*surveys))  # merge list of dicts
pprint(surveys)

{'284955270': 'Test Survey'}


### Download Survey

In [5]:
SURVEY_ID = 284955270

cached = [int(re.sub(r'[^\d]', '', x)) for x in glob.glob('data/*/')]
if SURVEY_ID not in cached:
    print(f"downloading survey {SURVEY_ID}")
    !python ../scripts/get_survey.py -e details -o data/$SURVEY_ID/details.json -t $TOKEN $SURVEY_ID
    !python ../scripts/get_survey.py -e responses -o data/$SURVEY_ID/responses -t $TOKEN $SURVEY_ID
    

In [6]:
!tree data

data
└── 284955270
    ├── details.json
    ├── responses
    │   ├── responses-00000.json
    │   ├── responses-00001.json
    │   ├── responses-00002.json
    │   └── responses-00003.json
    ├── test-survey-1.jpg
    ├── test-survey-2.jpg
    └── test-survey.pdf

2 directories, 8 files


## Preview Questions/Answers

In [7]:
with open(f'data/{SURVEY_ID}/details.json') as f:
    details = json.load(f)

questions = (
    jq
    .compile('.pages[].questions '
             '| map({name: .id, value: .headings[0].heading}) '
             '| from_entries')
    .input(details)
    .all()
)
questions = dict(ChainMap(*questions))  # merge list of dicts
questions = {k: re.sub(r'<[^>]*>', '', v) for k, v in questions.items()}
print("all questions")
pprint(questions)

# inspect question
question = 476164751
answers = (
    jq
    .compile('.pages[].questions[] '
             '| select(.id '
             f'| contains("{question}")) '
             '| .answers')
    .input(details)
    .all()
)
print(f'\ninspect question {question}')
pprint(answers)

all questions
{'476164746': 'We’d like to know how much you enjoy living in your current '
              'neighborhood. Thanks for sharing!',
 '476164747': 'Multiple Choice - Radio - Other Answer Choice',
 '476164748': 'Comment Box',
 '476164751': 'Multiple Choice - Radio',
 '476164752': 'Multiple Choice - Radio - Other Comment Box',
 '476164753': 'Multiple Choice - Checkboxes - Other Answer Choice',
 '476164754': 'Multiple Choice - Checkboxes',
 '476164755': 'Matrix Scale - Multiple Responses',
 '476190673': 'Contact Information',
 '476191619': 'Matrix Scale - Multiple Responses - Other'}

inspect question 476164751
[{'choices': [{'id': '3144764847',
               'position': 1,
               'quiz_options': {'score': 0},
               'text': 'A great deal',
               'visible': True},
              {'id': '3144764848',
               'position': 2,
               'quiz_options': {'score': 0},
               'text': 'A lot',
               'visible': True},
              {'id

## Transform to `DataFrame`

In [8]:
from spark_surveymonkey import _transform

df_flatten = _transform.flatten(spark, f'data/{SURVEY_ID}/responses')
df_interpret = _transform.interpret(df_flatten, f'data/{SURVEY_ID}/details.json')
df_pivot = _transform.pivot(df_interpret)

In [9]:
(
    _transform.interpret(df_flatten, f'data/{SURVEY_ID}/details.json')
    .sort(
        'response_id',
        'page_idx',
        'question_idx',
        F.coalesce('choice_id', 'row_id', 'other_id')
    )
    .withColumn('heading', F.col('heading').substr(1, 50))
    .withColumn('row', F.col('row'))#.substr(1, 50))
    .select(
        'response_id',
        'response_status',
        'page_idx',
        'question_idx',
        'question_id',
        'family',
        'heading',
        'row',
        'column',
        'value',
    )
#     .filter(F.col('value').isNull())  # should return 0 rows
    .show(20, False)
)


+-----------+---------------+--------+------------+-----------+---------------+--------------------------------------------------+---------+---------------------------------------------------------+----------------------------+
|response_id|response_status|page_idx|question_idx|question_id|family         |heading                                           |row      |column                                                   |value                       |
+-----------+---------------+--------+------------+-----------+---------------+--------------------------------------------------+---------+---------------------------------------------------------+----------------------------+
|11670854001|completed      |00001   |00002       |476164751  |single_choice  |Multiple Choice - Radio                           |null     |multiple_choice_radio                                    |A lot                       |
|11670854001|completed      |00001   |00003       |476164754  |multiple_choice|Multiple 

In [10]:
(
    _transform.pivot(df_interpret)
    .sort('response_id')
    .show(20, False)
)
pprint(df_pivot.columns)

+-----------+------------+---------------+---------------+------------+----------+---------+-------------+--------------+--------+---------+------------+---------+-------------------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------+----------+-------------------+-------------------+-----------------------------------------------------------------------+---------------------+------------------------------------------+-------------------------------------+-----------------------------------+-------------------------------------+-----------------------------------------+-----------------------------------------------+---------------------------------------+---------------------------------------------+---------------------------------------------------------+-------------------------------------------------------------+--------------